## 准备数据

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

    # Flatten images from 28x28 to 784-dimensional vectors
    x = x.reshape(-1, 28 * 28)  # Flatten to 784
    x_test = x_test.reshape(-1, 28 * 28)  # Flatten to 784

    return (x, y), (x_test, y_test)

In [12]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [13]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################

        # 第一层权重
        self.W1 = tf.Variable(tf.random.normal([28*28, 128]), name="W1")
        # 第一层偏置
        self.b1 = tf.Variable(tf.zeros([128]), name="b1")
        # 第二层权重
        self.W2 = tf.Variable(tf.random.normal([128, 10]), name="W2")
        # 第二层偏置
        self.b2 = tf.Variable(tf.zeros([10]), name="b2")
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################‘

        # 线性变换 (x * W1 + b1)
        h1 = tf.matmul(x, self.W1) + self.b1
        # 使用 ReLU 激活函数
        h1_relu = tf.nn.relu(h1)
        # 第二层线性变换
        logits = tf.matmul(h1_relu, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [14]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [15]:
train_data, test_data = mnist_dataset()
for epoch in range(200):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 123.81343 ; accuracy 0.12038333
epoch 1 : loss 106.462234 ; accuracy 0.12596667
epoch 2 : loss 93.778534 ; accuracy 0.12836666
epoch 3 : loss 84.7482 ; accuracy 0.12756667
epoch 4 : loss 78.2506 ; accuracy 0.12605
epoch 5 : loss 73.421555 ; accuracy 0.12516667
epoch 6 : loss 69.70016 ; accuracy 0.12503333
epoch 7 : loss 66.751526 ; accuracy 0.1266
epoch 8 : loss 64.34164 ; accuracy 0.12831667
epoch 9 : loss 62.310066 ; accuracy 0.13096666
epoch 10 : loss 60.56888 ; accuracy 0.13413334
epoch 11 : loss 59.047276 ; accuracy 0.13723333
epoch 12 : loss 57.688374 ; accuracy 0.14078334
epoch 13 : loss 56.453323 ; accuracy 0.1447
epoch 14 : loss 55.314697 ; accuracy 0.1485
epoch 15 : loss 54.2501 ; accuracy 0.15251666
epoch 16 : loss 53.245125 ; accuracy 0.15741667
epoch 17 : loss 52.289783 ; accuracy 0.1625
epoch 18 : loss 51.375534 ; accuracy 0.16705
epoch 19 : loss 50.49521 ; accuracy 0.17171666
epoch 20 : loss 49.643974 ; accuracy 0.17618333
epoch 21 : loss 48.81919 ; accura